In [1]:
import pandas as pd
import numpy as np
import requests
import json
import time
import datetime as dt

# Parameters
rivers_reference = pd.read_csv('RiverReferenceTable.csv')

sites = rivers_reference.USGS_ID.tolist()
string = ''

for i in sites:
    string = string+','+str(i)
sites=string[1:]

# URL to load json of data 
# Use format=rmd for readable URL
link = ('https://waterservices.usgs.gov/nwis/iv/?format=json&sites='+sites)
r=requests.get(link)
Geodata = json.loads(r.text)

Forecast_df = pd.DataFrame()

for i in range(len(Geodata['value']['timeSeries'])):
    lat = str(Geodata['value']['timeSeries'][i]['sourceInfo']['geoLocation']['geogLocation']['latitude'])
    long = str(Geodata['value']['timeSeries'][i]['sourceInfo']['geoLocation']['geogLocation']['longitude'])
    baseStation = Geodata['value']['timeSeries'][i]['sourceInfo']['siteName']
    # Search API for Lat Long Position
    r = requests.get('https://api.weather.gov/points/'+lat+','+long)
    json_data = r.json()

    print(baseStation)

    # Retrieve Forecast for Lat Long Position
    forecast_json = requests.get(json_data['properties']['forecast']).json()
    try: 
        if forecast_json['status'] == 500:
            continue
    except:
        KeyError
    forecast = forecast_json['properties']['periods']
    Forecast_df_temp = pd.DataFrame(forecast).drop(columns='number')
    Forecast_df_temp['Lat'] = lat
    Forecast_df_temp['Long'] = long

    Forecast_df_temp['Station'] = baseStation
    Forecast_df = Forecast_df.append(Forecast_df_temp,ignore_index=True)

Forecast_df.shape

E WALKER RV NR BRIDGEPORT, CA
E WALKER RV NR BRIDGEPORT, CA
E FK CARSON RV BLW MARKLEEVILLE CK NR MARKLEEVILLE
E FK CARSON RV BLW MARKLEEVILLE CK NR MARKLEEVILLE
LITTLE TRUCKEE R AB BOCA RES NR TRUCKEE CA
LITTLE TRUCKEE R AB BOCA RES NR TRUCKEE CA
TRUCKEE R A FARAD CA
TRUCKEE R A FARAD CA
TRUCKEE R A FARAD CA
SAN LORENZO R A BIG TREES CA
SAN LORENZO R A BIG TREES CA
SAN LORENZO R A SANTA CRUZ CA
SAN LORENZO R A SANTA CRUZ CA
PIT R BL PIT NO 1 PH NR FALL RIVER MILLS CA
PIT R BL PIT NO 1 PH NR FALL RIVER MILLS CA
HAT C NR HAT CREEK CA
HAT C NR HAT CREEK CA
HAT C NR HAT CREEK CA
N YUBA R BL GOODYEARS BAR CA
N YUBA R BL GOODYEARS BAR CA
YUBA R NR MARYSVILLE CA
YUBA R NR MARYSVILLE CA
NF AMERICAN R A NORTH FORK DAM CA
NF AMERICAN R A NORTH FORK DAM CA
PUTAH C NR WINTERS CA
PUTAH C NR WINTERS CA
RUSSIAN R A HACIENDA BRIDGE NR GUERNEVILLE CA
RUSSIAN R A HACIENDA BRIDGE NR GUERNEVILLE CA
RUSSIAN R A HACIENDA BRIDGE NR GUERNEVILLE CA
RUSSIAN R A HACIENDA BRIDGE NR GUERNEVILLE CA
RUSSIAN R A HAC

(1022, 15)

In [9]:
df = Forecast_df.drop_duplicates()
df.startTime = pd.to_datetime(df.startTime,utc=True)
df.endTime = pd.to_datetime(df.endTime,utc=True)
df.drop(columns=['temperatureTrend','icon'],inplace=True)
df.head()

/Users/scottstandring/opt/miniconda3/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/Users/scottstandring/opt/miniconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,name,startTime,endTime,isDaytime,temperature,temperatureUnit,windSpeed,windDirection,shortForecast,detailedForecast,Lat,Long,Station
0,This Afternoon,2022-03-11 21:00:00+00:00,2022-03-12 02:00:00+00:00,True,49,F,5 to 10 mph,NW,Sunny,"Sunny, with a high near 49. Northwest wind 5 t...",38.3276958,-119.2148759,"E WALKER RV NR BRIDGEPORT, CA"
1,Tonight,2022-03-12 02:00:00+00:00,2022-03-12 14:00:00+00:00,False,21,F,0 to 10 mph,SW,Mostly Clear,"Mostly clear, with a low around 21. Southwest ...",38.3276958,-119.2148759,"E WALKER RV NR BRIDGEPORT, CA"
2,Saturday,2022-03-12 14:00:00+00:00,2022-03-13 02:00:00+00:00,True,53,F,5 to 15 mph,SW,Sunny,"Sunny, with a high near 53. Southwest wind 5 t...",38.3276958,-119.2148759,"E WALKER RV NR BRIDGEPORT, CA"
3,Saturday Night,2022-03-13 02:00:00+00:00,2022-03-13 13:00:00+00:00,False,26,F,10 to 15 mph,SW,Partly Cloudy,"Partly cloudy, with a low around 26. Southwest...",38.3276958,-119.2148759,"E WALKER RV NR BRIDGEPORT, CA"
4,Sunday,2022-03-13 13:00:00+00:00,2022-03-14 01:00:00+00:00,True,49,F,15 mph,W,Mostly Sunny,"Mostly sunny, with a high near 49. West wind a...",38.3276958,-119.2148759,"E WALKER RV NR BRIDGEPORT, CA"


In [10]:
import re

def LightRain(s):
    if re.search('Slight Chance Rain',s):
        value = 1
    elif re.search('Light Rain',s):
        value = 1  
    else: value = 0
    return value

def Rain(s):
    if re.search('Rain Showers',s):
        value = 1
    else: value = 0
    return value

def HeavyRain(s):
    if re.search('Heavy Rain',s):
        value = 1
    else: value = 0
    return value

def WindTranslate(s):
    s = ' '+s
    l = len(s)-4
    value = s[l-2:l]
    return int(value)
    

df['LightRain'] = df.shortForecast.apply(lambda x: LightRain(x))
df['Rain'] = df.shortForecast.apply(lambda x: Rain(x))
df['HeavyRain'] = df.shortForecast.apply(lambda x: HeavyRain(x))
df['WindMph'] = df.windSpeed.apply(lambda x: WindTranslate(x))

/var/folders/0d/58w421rs1hvdqjc9ztxvvbcm0000gn/T/ipykernel_13185/2970147979.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LightRain'] = df.shortForecast.apply(lambda x: LightRain(x))
/var/folders/0d/58w421rs1hvdqjc9ztxvvbcm0000gn/T/ipykernel_13185/2970147979.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Rain'] = df.shortForecast.apply(lambda x: Rain(x))
/var/folders/0d/58w421rs1hvdqjc9ztxvvbcm0000gn/T/ipykernel_13185/2970147979.py:32: SettingWithCopyWarning: 
A value is trying to be se

In [41]:
WeatherForecast = df[['Station','name','startTime','endTime','temperature','windDirection','WindMph','LightRain','Rain','HeavyRain']]
WeatherForecast = WeatherForecast.set_index('Station').join(rivers_reference[['USGS Name','Name']].set_index('USGS Name'))
WeatherForecast = WeatherForecast.reset_index().rename(columns={'index':'Station','Name':'StationName'})
WeatherForecast['Date_Name'] = WeatherForecast.startTime.dt.date.astype(str)+'_'+WeatherForecast.StationName
WeatherForecast.head()

,Station,name,startTime,endTime,temperature,windDirection,WindMph,LightRain,Rain,HeavyRain,StationName,Date_Name
0,E FK CARSON RV BLW MARKLEEVILLE CK NR MARKLEEV...,This Afternoon,2022-03-11 21:00:00+00:00,2022-03-12 02:00:00+00:00,54,W,5,0,0,0,East Fork Carson,2022-03-11_East Fork Carson
1,E FK CARSON RV BLW MARKLEEVILLE CK NR MARKLEEV...,Tonight,2022-03-12 02:00:00+00:00,2022-03-12 14:00:00+00:00,26,S,10,0,0,0,East Fork Carson,2022-03-12_East Fork Carson
2,E FK CARSON RV BLW MARKLEEVILLE CK NR MARKLEEV...,Saturday,2022-03-12 14:00:00+00:00,2022-03-13 02:00:00+00:00,56,S,15,0,0,0,East Fork Carson,2022-03-12_East Fork Carson
3,E FK CARSON RV BLW MARKLEEVILLE CK NR MARKLEEV...,Saturday Night,2022-03-13 02:00:00+00:00,2022-03-13 13:00:00+00:00,31,SW,15,0,0,0,East Fork Carson,2022-03-13_East Fork Carson
4,E FK CARSON RV BLW MARKLEEVILLE CK NR MARKLEEV...,Sunday,2022-03-13 13:00:00+00:00,2022-03-14 01:00:00+00:00,50,SW,15,0,0,0,East Fork Carson,2022-03-13_East Fork Carson


In [34]:
Weather = pd.read_csv('CurrentPrecip.csv').drop(columns=['Unnamed: 0'])
Weather = Weather.drop_duplicates()
Weather['Date_Name'] = Weather.Date+'_'+Weather.Name
Weather.head()

,Date,Precip,Name,Date_Name
0,2021-11-27,0.0,Russian River HaciendaBridge,2021-11-27_Russian River HaciendaBridge
2,2021-11-28,0.0,Russian River HaciendaBridge,2021-11-28_Russian River HaciendaBridge
4,2021-11-29,0.0,Russian River HaciendaBridge,2021-11-29_Russian River HaciendaBridge
6,2021-11-30,0.0,Russian River HaciendaBridge,2021-11-30_Russian River HaciendaBridge
8,2021-12-01,0.0,Russian River HaciendaBridge,2021-12-01_Russian River HaciendaBridge
